In [5]:
%autoreload 2
from pymcts.uct import UCTNode
from pymcts.game.trivial import TrivialState
from pymcts.game.tic_tac_toe import TicTacToeState

In [19]:
win_state = TrivialState()
loss_state = TrivialState(result={1: 0})
root_state = TrivialState(result=None, moves={1: win_state, 2: loss_state})

root = UCTNode(root_state)
for _ in range(100):
    root.mc_round()

root

UCTNode(M: None, P1, Wins/Visits: 94.0/100.0, # Untried: 0, [
    UCTNode(M: 2, P1, Wins/Visits: 0.0/6.0, # Untried: 0),
    UCTNode(M: 1, P1, Wins/Visits: 94.0/94.0, # Untried: 0)
])

Above we see a trivial two-stage game where Player 1 always 'wins' following taking move 1, and always loses after taking move 2.

We then set up a UCT tree and run some iterations. One consequence of the UCT algorithm here is that an always-losing node will get revisited with exponentially declining frequency as the number of iterations increase. This shows that UCT focuses its visits (during the 'selection' part of MCTS) on subtrees that perform well, and hence searches deeper into parts of the game tree that see stronger play.

In [15]:
s = TicTacToeState()

root = UCTNode(s)
for _ in range(int(1E5)):
    root.mc_round()
root.wins / root.visits

0.422095

Here we repeat the exercise with Tic Tac Toe. We know that perfect play always results in a tie (score = 0.5); in general, UCB scores should underestimate the true value of a node and converge from below as the number of iterations rises.

In [9]:
root.best_move()

(1, 1)

As expected, this tree identifies that the best move for the first player is to go in the center.